In [103]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import lightgbm as lgb
import statsmodels.api as sm
import warnings
import shap
from sklearn.model_selection import StratifiedKFold
from pycaret.regression import *
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.tree import ExtraTreeRegressor
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import KNNImputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostClassifier, Pool
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import AdaBoostRegressor
from catboost import CatBoostRegressor
from catboost import CatBoostClassifier
from sklearn.metrics import mean_squared_error
from pycaret.regression import *
from sklearn.impute import KNNImputer

In [104]:
warnings.simplefilter(action='ignore')
pd.set_option('display.max_columns', None)

In [164]:
train_data = pd.read_csv("./train.csv")
train_data

,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,TRAIN_000,1,0.533433,2022-06-13 5:14,T050304,A_31,NaN,NaN,NaN,NaN,...,39.34,40.89,32.56,34.09,77.77,NaN,NaN,NaN,NaN,NaN
1,TRAIN_001,2,0.541819,2022-06-13 5:22,T050307,A_31,NaN,NaN,NaN,NaN,...,38.89,42.82,43.92,35.34,72.55,NaN,NaN,NaN,NaN,NaN
2,TRAIN_002,1,0.531267,2022-06-13 5:30,T050304,A_31,NaN,NaN,NaN,NaN,...,39.19,36.65,42.47,36.53,78.35,NaN,NaN,NaN,NaN,NaN
3,TRAIN_003,2,0.537325,2022-06-13 5:39,T050307,A_31,NaN,NaN,NaN,NaN,...,37.74,39.17,52.17,30.58,71.78,NaN,NaN,NaN,NaN,NaN
4,TRAIN_004,1,0.531590,2022-06-13 5:47,T050304,A_31,NaN,NaN,NaN,NaN,...,38.70,41.89,46.93,33.09,76.97,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,TRAIN_593,1,0.526546,2022-09-08 14:30,T100306,T_31,2.0,95.0,0.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
594,TRAIN_594,0,0.524022,2022-09-08 22:38,T050304,A_31,NaN,NaN,NaN,NaN,...,49.47,53.07,50.89,55.10,66.49,1.0,NaN,NaN,NaN,NaN
595,TRAIN_595,0,0.521289,2022-09-08 22:47,T050304,A_31,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
596,TRAIN_596,1,0.531375,2022-09-08 14:38,T100304,O_31,40.0,94.0,0.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [163]:
train_data['YEAR'] =  pd.to_datetime(train_data['TIMESTAMP']).dt.year
train_data['MONTH'] =  pd.to_datetime(train_data['TIMESTAMP']).dt.month
train_data['DAY'] =  pd.to_datetime(train_data['TIMESTAMP']).dt.day
train_data['HOUR'] =  pd.to_datetime(train_data['TIMESTAMP']).dt.hour
train_data['MINUTE'] =  pd.to_datetime(train_data['TIMESTAMP']).dt.minute
train_data['DAY_OF_WEEK'] =  pd.to_datetime(train_data['TIMESTAMP']).dt.day_name()

In [165]:
colls = []
for coll in train_data :
    if 'X' in coll and train_data[coll].isna().sum() == 598 :
        colls.append(coll)
print(len(colls))

82


In [166]:
train_data.drop(colls, inplace = True, axis = 1)

In [94]:
train_data = train_data.drop(['MONTH', 'MINUTE', 'DAY', 'YEAR', 'DAY_OF_WEEK', 'HOUR'], axis = 1)

In [167]:
train_data.loc[train_data.PRODUCT_CODE == 'A_31','PRODUCT_CODE'] = 1
train_data.loc[train_data.PRODUCT_CODE == 'T_31','PRODUCT_CODE'] = 2
train_data.loc[train_data.PRODUCT_CODE == 'O_31','PRODUCT_CODE'] = 3

train_data.drop(['PRODUCT_ID', 'TIMESTAMP'], axis = 1, inplace = True)
train_data

,Y_Class,Y_Quality,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,...,X_2862,X_2863,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871
0,1,0.533433,T050304,1,NaN,NaN,NaN,NaN,NaN,NaN,...,189.0,383.0,368.296296,353.0,39.34,40.89,32.56,34.09,77.77,NaN
1,2,0.541819,T050307,1,NaN,NaN,NaN,NaN,NaN,NaN,...,185.6,383.0,367.735849,353.0,38.89,42.82,43.92,35.34,72.55,NaN
2,1,0.531267,T050304,1,NaN,NaN,NaN,NaN,NaN,NaN,...,165.5,383.0,367.320755,353.0,39.19,36.65,42.47,36.53,78.35,NaN
3,2,0.537325,T050307,1,NaN,NaN,NaN,NaN,NaN,NaN,...,165.8,384.0,369.188679,353.0,37.74,39.17,52.17,30.58,71.78,NaN
4,1,0.531590,T050304,1,NaN,NaN,NaN,NaN,NaN,NaN,...,182.6,383.0,367.351852,352.0,38.70,41.89,46.93,33.09,76.97,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,1,0.526546,T100306,2,2.0,95.0,0.0,45.0,10.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
594,0,0.524022,T050304,1,NaN,NaN,NaN,NaN,NaN,NaN,...,168.7,384.0,369.811321,353.0,49.47,53.07,50.89,55.10,66.49,1.0
595,0,0.521289,T050304,1,NaN,NaN,NaN,NaN,NaN,NaN,...,156.6,383.0,367.018868,352.0,NaN,NaN,NaN,NaN,NaN,1.0
596,1,0.531375,T100304,3,40.0,94.0,0.0,45.0,11.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [168]:
T05 = train_data[(train_data['LINE'] == 'T050304')|(train_data['LINE'] == 'T050307')]
T01 = train_data[(train_data['LINE'] == 'T010305')|(train_data['LINE'] == 'T010306')]
T10 = train_data[(train_data['LINE'] == 'T100304')|(train_data['LINE'] == 'T100306')]

In [169]:
T05.drop(['LINE'], axis = 1, inplace = True)
T01.drop(['LINE'], axis = 1, inplace = True)
T10.drop(['LINE'], axis = 1, inplace = True)

In [170]:
T05_col = []
for col in T05 :
    if 'X' in col and T05[col].isna().sum() == len(T05) :
        T05_col.append(col)
print(len(T05_col))

T05.drop(T05_col, inplace = True, axis = 1)

806


In [171]:
T01_col = []
for col in T01 :
    if 'X' in col and T01[col].isna().sum() == len(T01) :
        T01_col.append(col)
print(len(T01_col))

T01.drop(T01_col, inplace = True, axis = 1)

1899


In [172]:
T10_col = []
for col in T10 :
    if 'X' in col and T10[col].isna().sum() == len(T10) :
        T10_col.append(col)
print(len(T10_col))

T10.drop(T10_col, inplace = True, axis = 1)

2116


In [161]:
T05 = T05.fillna(T05.mean())
T01 = T01.fillna(T01.mean())
T10 = T10.fillna(T10.mean())

In [173]:
T05 = T05.fillna(0)
T01 = T01.fillna(0)
T10 = T10.fillna(0)

In [89]:
zero_one = (train_data[train_data['Y_Class'] == 1]['Y_Quality'].min() + train_data[train_data['Y_Class'] == 0]['Y_Quality'].max())/2
one_two = (train_data[train_data['Y_Class'] == 2]['Y_Quality'].min() + train_data[train_data['Y_Class'] == 1]['Y_Quality'].max())/2
print(zero_one, one_two)

0.5250761905000001 0.5348968255


# Model

# T05

In [174]:
T05_class = T05['Y_Class']
T05_target = T05['Y_Quality']
T05_data = T05.drop(['Y_Class', 'Y_Quality'], axis = 1)
T05_data

,PRODUCT_CODE,X_128,X_129,X_130,X_131,X_132,X_133,X_134,X_135,X_136,...,X_2862,X_2863,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871
0,1,7813.0,7813.0,0.0,0.0,0.19,0.20,0.19,0.2,228.0,...,189.0,383.0,368.296296,353.0,39.34,40.89,32.56,34.09,77.77,0.0
1,1,0.0,0.0,19854.0,19854.0,0.20,0.21,0.20,0.2,413.0,...,185.6,383.0,367.735849,353.0,38.89,42.82,43.92,35.34,72.55,0.0
2,1,7815.0,7815.0,0.0,0.0,0.19,0.20,0.19,0.2,228.0,...,165.5,383.0,367.320755,353.0,39.19,36.65,42.47,36.53,78.35,0.0
3,1,0.0,0.0,19856.0,19856.0,0.20,0.21,0.20,0.2,414.0,...,165.8,384.0,369.188679,353.0,37.74,39.17,52.17,30.58,71.78,0.0
4,1,7817.0,7817.0,0.0,0.0,0.19,0.20,0.18,0.2,228.0,...,182.6,383.0,367.351852,352.0,38.70,41.89,46.93,33.09,76.97,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581,1,11864.0,11864.0,0.0,0.0,0.19,0.20,0.19,0.2,230.0,...,176.0,384.0,367.333333,352.0,0.00,0.00,0.00,0.00,0.00,1.0
582,1,11898.0,11898.0,0.0,0.0,0.18,0.20,0.18,0.2,230.0,...,174.0,384.0,367.037037,352.0,50.88,53.23,52.44,56.28,66.83,1.0
583,1,11920.0,11920.0,0.0,0.0,0.19,0.20,0.19,0.2,230.0,...,181.6,394.0,371.943396,353.0,51.71,59.64,54.61,57.05,63.18,1.0
594,1,14810.0,14810.0,0.0,0.0,0.19,0.20,0.19,0.2,304.0,...,168.7,384.0,369.811321,353.0,49.47,53.07,50.89,55.10,66.49,1.0


In [175]:
#13
setup_rgs = setup(data = T05_data, target=T05_target, session_id = 414)
top5 = compare_models(sort='MSE', n_select=5)

,Description,Value
0,Session id,414
1,Target,Y_Quality
2,Target type,Regression
3,Original data shape,"(120, 1989)"
4,Transformed data shape,"(120, 1989)"
5,Transformed train set shape,"(84, 1989)"
6,Transformed test set shape,"(36, 1989)"
7,Numeric features,1988
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,0.0045,0.0000,0.0062,0.4151,0.0040,0.0083,0.0490
catboost,CatBoost Regressor,0.0049,0.0001,0.0071,0.3516,0.0046,0.0092,0.1570
ridge,Ridge Regression,0.0080,0.0001,0.0107,-0.8038,0.0070,0.0150,0.0370
xgboost,Extreme Gradient Boosting,0.0049,0.0001,0.0068,0.3718,0.0044,0.0091,0.0450
llar,Lasso Least Angle Regression,0.0078,0.0001,0.0103,-0.4158,0.0067,0.0145,0.0400
lr,Linear Regression,0.0073,0.0001,0.0099,-0.3381,0.0065,0.0137,0.6220
en,Elastic Net,0.0074,0.0001,0.0098,-0.3097,0.0064,0.0137,0.0380
lasso,Lasso Regression,0.0074,0.0001,0.0099,-0.3313,0.0064,0.0139,0.2240
huber,Huber Regressor,0.0074,0.0001,0.0099,-0.3938,0.0064,0.0138,0.0440
knn,K Neighbors Regressor,0.0072,0.0001,0.0102,-0.4578,0.0067,0.0133,0.0440


Processing:   0%|          | 0/89 [00:00<?, ?it/s]

In [176]:
T05_total_models = []

T05_model_lightgbm = create_model('lightgbm', fold = 5)
T05_total_models.append(T05_model_lightgbm)

T05_model_rf = create_model('rf', fold = 5)
T05_total_models.append(T05_model_rf)

T05_model_gbr = create_model('gbr', fold = 5)
T05_total_models.append(T05_model_gbr)

T05_model_et = create_model('et', fold = 5)
T05_total_models.append(T05_model_et)

T05_model_catboost = create_model('catboost', fold = 5)
T05_total_models.append(T05_model_catboost)

T05_model_xgb = create_model('xgboost', fold = 5)
T05_total_models.append(T05_model_xgb)

T05_model_ada = create_model('ada', fold = 5)

T05_total_models.append(T05_model_ada)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0061,0.0001,0.0077,-0.1622,0.0051,0.0116
1,0.0050,0.0001,0.0074,0.4889,0.0048,0.0094
2,0.0045,0.0001,0.0071,0.3635,0.0046,0.0084
3,0.0077,0.0001,0.0092,0.2814,0.0060,0.0143
4,0.0080,0.0002,0.0122,0.1516,0.0079,0.0147
Mean,0.0063,0.0001,0.0087,0.2246,0.0057,0.0117
Std,0.0014,0.0000,0.0019,0.2224,0.0012,0.0025


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0053,0.0000,0.0062,0.2497,0.0041,0.0101
1,0.0048,0.0000,0.0064,0.6199,0.0042,0.0089
2,0.0048,0.0001,0.0073,0.3278,0.0047,0.0090
3,0.0061,0.0001,0.0078,0.4836,0.0050,0.0114
4,0.0062,0.0001,0.0099,0.4438,0.0064,0.0113
Mean,0.0054,0.0001,0.0075,0.4250,0.0049,0.0102
Std,0.0006,0.0000,0.0013,0.1280,0.0008,0.0011


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0042,0.0000,0.0049,0.5252,0.0032,0.0080
1,0.0037,0.0000,0.0046,0.8018,0.0030,0.0071
2,0.0035,0.0000,0.0049,0.7045,0.0032,0.0066
3,0.0072,0.0001,0.0099,0.1688,0.0064,0.0134
4,0.0057,0.0001,0.0096,0.4827,0.0062,0.0104
Mean,0.0049,0.0001,0.0068,0.5366,0.0044,0.0091
Std,0.0014,0.0000,0.0024,0.2176,0.0015,0.0025


...

Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0036,0.0000,0.0046,0.5917,0.0030,0.0069
1,0.0045,0.0000,0.0062,0.6445,0.0040,0.0085
2,0.0045,0.0001,0.0075,0.2954,0.0049,0.0083
3,0.0055,0.0000,0.0069,0.5906,0.0045,0.0103
4,0.0069,0.0001,0.0105,0.3816,0.0067,0.0127
Mean,0.0050,0.0001,0.0071,0.5008,0.0046,0.0093
Std,0.0011,0.0000,0.0019,0.1367,0.0012,0.0020


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0049,0.0000,0.0062,0.2506,0.0041,0.0094
1,0.0048,0.0000,0.0063,0.6363,0.0041,0.0091
2,0.0048,0.0001,0.0072,0.3486,0.0047,0.0090
3,0.0069,0.0001,0.0089,0.3259,0.0058,0.0127
4,0.0057,0.0001,0.0089,0.5570,0.0057,0.0105
Mean,0.0054,0.0001,0.0075,0.4237,0.0049,0.0101
Std,0.0008,0.0000,0.0012,0.1471,0.0007,0.0014


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [177]:
T05_model_lightgbm = tune_model(T05_model_lightgbm, fold=5, choose_better = True)

T05_model_rf = tune_model(T05_model_rf, fold=5, choose_better = True)

T05_model_gbr = tune_model(T05_model_gbr, fold=5, choose_better = True)

T05_model_et = tune_model(T05_model_et, fold=5, choose_better = True)

T05_model_catboost = tune_model(T05_model_catboost, fold=5, choose_better = True)

T05_model_xgb = tune_model(T05_model_xgb, fold=5, choose_better = True)

T05_model_ada = tune_model(T05_model_ada, fold=5, choose_better = True)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0074,0.0001,0.0088,-0.5051,0.0058,0.0141
1,0.0078,0.0001,0.0105,-0.0190,0.0068,0.0147
2,0.0062,0.0001,0.0091,-0.0304,0.0059,0.0117
3,0.0100,0.0002,0.0123,-0.2856,0.0080,0.0184
4,0.0079,0.0002,0.0135,-0.0332,0.0087,0.0145
Mean,0.0079,0.0001,0.0108,-0.1747,0.0070,0.0147
Std,0.0012,0.0000,0.0018,0.1932,0.0012,0.0022


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0068,0.0001,0.0087,-0.4848,0.0057,0.0130
1,0.0057,0.0001,0.0085,0.3312,0.0055,0.0107
2,0.0049,0.0001,0.0084,0.1183,0.0054,0.0091
3,0.0083,0.0001,0.0114,-0.1067,0.0074,0.0153
4,0.0063,0.0001,0.0117,0.2231,0.0075,0.0114
Mean,0.0064,0.0001,0.0097,0.0162,0.0063,0.0119
Std,0.0011,0.0000,0.0015,0.2893,0.0009,0.0021


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

...

Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0049,0.0000,0.0059,0.3300,0.0038,0.0092
1,0.0048,0.0000,0.0062,0.6449,0.0040,0.0090
2,0.0045,0.0001,0.0071,0.3604,0.0046,0.0083
3,0.0075,0.0001,0.0098,0.1830,0.0063,0.0139
4,0.0057,0.0001,0.0094,0.5045,0.0060,0.0104
Mean,0.0055,0.0001,0.0077,0.4046,0.0050,0.0102
Std,0.0011,0.0000,0.0016,0.1577,0.0010,0.0020


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [178]:
T05_tuned_models = [T05_model_lightgbm, T05_model_rf, T05_model_gbr, T05_model_et, T05_model_catboost, T05_model_xgb, T05_model_ada]
T05_tuned_models

[LGBMRegressor(random_state=414),
 RandomForestRegressor(n_jobs=-1, random_state=414),
 GradientBoostingRegressor(random_state=414),
 ExtraTreesRegressor(n_jobs=-1, random_state=414),
 XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=-1, num_parallel_tree=1, predictor='auto', random_state=414,
              reg_alpha=0, reg_lambda=1, ...),
 AdaBoostRegressor(random_state=414)]

In [179]:
for model in T05_tuned_models:
    display(predict_model(model))

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,0.0044,0.0000,0.0060,0.6881,0.0039,0.0083


,PRODUCT_CODE,X_128,X_129,X_130,X_131,X_132,X_133,X_134,X_135,X_136,...,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,Y_Quality,prediction_label
84,1,3538.0,3538.0,0.0,0.0,0.20,0.21,0.20,0.2,107.0,...,363.641510,342.0,49.790001,53.000000,53.740002,50.090000,62.770000,1.0,0.521341,0.520427
85,1,0.0,0.0,829.0,829.0,0.19,0.20,0.19,0.2,58.0,...,367.905670,352.0,51.099998,52.730000,56.139999,50.799999,62.950001,1.0,0.524090,0.521892
86,1,24448.0,24448.0,0.0,0.0,0.19,0.21,0.19,0.2,811.0,...,367.555542,352.0,54.529999,59.330002,50.849998,55.330002,65.870003,1.0,0.520856,0.518730
87,1,11772.0,11772.0,0.0,0.0,0.19,0.20,0.19,0.2,228.0,...,370.648163,353.0,47.189999,53.889999,52.970001,56.320000,66.690002,1.0,0.522460,0.525691
88,1,25323.0,25323.0,0.0,0.0,0.20,0.21,0.20,0.2,653.0,...,368.830200,352.0,53.490002,55.590000,60.000000,58.180000,66.790001,1.0,0.538629,0.529223
89,1,17441.0,17441.0,0.0,0.0,0.20,0.21,0.20,0.2,571.0,...,368.528290,352.0,60.240002,56.160000,54.520000,56.419998,66.849998,1.0,0.515083,0.528971
90,1,11743.0,11743.0,0.0,0.0,0.20,0.21,0.19,0.2,421.0,...,367.166656,352.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.535405,0.526875
91,1,0.0,0.0,6473.0,6473.0,0.20,0.21,0.20,0.2,171.0,...,369.358490,353.0,59.380001,57.130001,50.150002,57.169998,64.010002,1.0,0.544584,0.535766
92,1,0.0,0.0,1561.0,1561.0,0.20,0.21,0.20,0.2,67.0,...,368.358490,353.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.543957,0.541874
93,1,2947.0,2947.0,0.0,0.0,0.20,0.21,0.19,0.2,123.0,...,367.716980,352.0,48.279999,51.250000,55.430000,56.950001,67.440002,1.0,0.519451,0.526828


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Random Forest Regressor,0.0053,0.0000,0.0062,0.6588,0.0041,0.0100


,PRODUCT_CODE,X_128,X_129,X_130,X_131,X_132,X_133,X_134,X_135,X_136,...,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,Y_Quality,prediction_label
84,1,3538.0,3538.0,0.0,0.0,0.20,0.21,0.20,0.2,107.0,...,363.641510,342.0,49.790001,53.000000,53.740002,50.090000,62.770000,1.0,0.521341,0.526781
85,1,0.0,0.0,829.0,829.0,0.19,0.20,0.19,0.2,58.0,...,367.905670,352.0,51.099998,52.730000,56.139999,50.799999,62.950001,1.0,0.524090,0.524517
86,1,24448.0,24448.0,0.0,0.0,0.19,0.21,0.19,0.2,811.0,...,367.555542,352.0,54.529999,59.330002,50.849998,55.330002,65.870003,1.0,0.520856,0.525133
87,1,11772.0,11772.0,0.0,0.0,0.19,0.20,0.19,0.2,228.0,...,370.648163,353.0,47.189999,53.889999,52.970001,56.320000,66.690002,1.0,0.522460,0.527745
88,1,25323.0,25323.0,0.0,0.0,0.20,0.21,0.20,0.2,653.0,...,368.830200,352.0,53.490002,55.590000,60.000000,58.180000,66.790001,1.0,0.538629,0.530892
89,1,17441.0,17441.0,0.0,0.0,0.20,0.21,0.20,0.2,571.0,...,368.528290,352.0,60.240002,56.160000,54.520000,56.419998,66.849998,1.0,0.515083,0.526462
90,1,11743.0,11743.0,0.0,0.0,0.20,0.21,0.19,0.2,421.0,...,367.166656,352.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.535405,0.528381
91,1,0.0,0.0,6473.0,6473.0,0.20,0.21,0.20,0.2,171.0,...,369.358490,353.0,59.380001,57.130001,50.150002,57.169998,64.010002,1.0,0.544584,0.538198
92,1,0.0,0.0,1561.0,1561.0,0.20,0.21,0.20,0.2,67.0,...,368.358490,353.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.543957,0.535479
93,1,2947.0,2947.0,0.0,0.0,0.20,0.21,0.19,0.2,123.0,...,367.716980,352.0,48.279999,51.250000,55.430000,56.950001,67.440002,1.0,0.519451,0.526681


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Gradient Boosting Regressor,0.0046,0.0000,0.0055,0.7359,0.0036,0.0086


...

,PRODUCT_CODE,X_128,X_129,X_130,X_131,X_132,X_133,X_134,X_135,X_136,...,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,Y_Quality,prediction_label
84,1,3538.0,3538.0,0.0,0.0,0.20,0.21,0.20,0.2,107.0,...,363.641510,342.0,49.790001,53.000000,53.740002,50.090000,62.770000,1.0,0.521341,0.526073
85,1,0.0,0.0,829.0,829.0,0.19,0.20,0.19,0.2,58.0,...,367.905670,352.0,51.099998,52.730000,56.139999,50.799999,62.950001,1.0,0.524090,0.522028
86,1,24448.0,24448.0,0.0,0.0,0.19,0.21,0.19,0.2,811.0,...,367.555542,352.0,54.529999,59.330002,50.849998,55.330002,65.870003,1.0,0.520856,0.524568
87,1,11772.0,11772.0,0.0,0.0,0.19,0.20,0.19,0.2,228.0,...,370.648163,353.0,47.189999,53.889999,52.970001,56.320000,66.690002,1.0,0.522460,0.526380
88,1,25323.0,25323.0,0.0,0.0,0.20,0.21,0.20,0.2,653.0,...,368.830200,352.0,53.490002,55.590000,60.000000,58.180000,66.790001,1.0,0.538629,0.532191
89,1,17441.0,17441.0,0.0,0.0,0.20,0.21,0.20,0.2,571.0,...,368.528290,352.0,60.240002,56.160000,54.520000,56.419998,66.849998,1.0,0.515083,0.530310
90,1,11743.0,11743.0,0.0,0.0,0.20,0.21,0.19,0.2,421.0,...,367.166656,352.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.535405,0.527653
91,1,0.0,0.0,6473.0,6473.0,0.20,0.21,0.20,0.2,171.0,...,369.358490,353.0,59.380001,57.130001,50.150002,57.169998,64.010002,1.0,0.544584,0.536610
92,1,0.0,0.0,1561.0,1561.0,0.20,0.21,0.20,0.2,67.0,...,368.358490,353.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.543957,0.533326
93,1,2947.0,2947.0,0.0,0.0,0.20,0.21,0.19,0.2,123.0,...,367.716980,352.0,48.279999,51.250000,55.430000,56.950001,67.440002,1.0,0.519451,0.525726


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extreme Gradient Boosting,0.0052,0.0000,0.0068,0.5939,0.0044,0.0098


,PRODUCT_CODE,X_128,X_129,X_130,X_131,X_132,X_133,X_134,X_135,X_136,...,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,Y_Quality,prediction_label
84,1,3538.0,3538.0,0.0,0.0,0.20,0.21,0.20,0.2,107.0,...,363.641510,342.0,49.790001,53.000000,53.740002,50.090000,62.770000,1.0,0.521341,0.528105
85,1,0.0,0.0,829.0,829.0,0.19,0.20,0.19,0.2,58.0,...,367.905670,352.0,51.099998,52.730000,56.139999,50.799999,62.950001,1.0,0.524090,0.520254
86,1,24448.0,24448.0,0.0,0.0,0.19,0.21,0.19,0.2,811.0,...,367.555542,352.0,54.529999,59.330002,50.849998,55.330002,65.870003,1.0,0.520856,0.525075
87,1,11772.0,11772.0,0.0,0.0,0.19,0.20,0.19,0.2,228.0,...,370.648163,353.0,47.189999,53.889999,52.970001,56.320000,66.690002,1.0,0.522460,0.519737
88,1,25323.0,25323.0,0.0,0.0,0.20,0.21,0.20,0.2,653.0,...,368.830200,352.0,53.490002,55.590000,60.000000,58.180000,66.790001,1.0,0.538629,0.532075
89,1,17441.0,17441.0,0.0,0.0,0.20,0.21,0.20,0.2,571.0,...,368.528290,352.0,60.240002,56.160000,54.520000,56.419998,66.849998,1.0,0.515083,0.525621
90,1,11743.0,11743.0,0.0,0.0,0.20,0.21,0.19,0.2,421.0,...,367.166656,352.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.535405,0.522150
91,1,0.0,0.0,6473.0,6473.0,0.20,0.21,0.20,0.2,171.0,...,369.358490,353.0,59.380001,57.130001,50.150002,57.169998,64.010002,1.0,0.544584,0.536692
92,1,0.0,0.0,1561.0,1561.0,0.20,0.21,0.20,0.2,67.0,...,368.358490,353.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.543957,0.529910
93,1,2947.0,2947.0,0.0,0.0,0.20,0.21,0.19,0.2,123.0,...,367.716980,352.0,48.279999,51.250000,55.430000,56.950001,67.440002,1.0,0.519451,0.523734


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,AdaBoost Regressor,0.0052,0.0000,0.0062,0.6620,0.0041,0.0099


,PRODUCT_CODE,X_128,X_129,X_130,X_131,X_132,X_133,X_134,X_135,X_136,...,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,Y_Quality,prediction_label
84,1,3538.0,3538.0,0.0,0.0,0.20,0.21,0.20,0.2,107.0,...,363.641510,342.0,49.790001,53.000000,53.740002,50.090000,62.770000,1.0,0.521341,0.525559
85,1,0.0,0.0,829.0,829.0,0.19,0.20,0.19,0.2,58.0,...,367.905670,352.0,51.099998,52.730000,56.139999,50.799999,62.950001,1.0,0.524090,0.523427
86,1,24448.0,24448.0,0.0,0.0,0.19,0.21,0.19,0.2,811.0,...,367.555542,352.0,54.529999,59.330002,50.849998,55.330002,65.870003,1.0,0.520856,0.522877
87,1,11772.0,11772.0,0.0,0.0,0.19,0.20,0.19,0.2,228.0,...,370.648163,353.0,47.189999,53.889999,52.970001,56.320000,66.690002,1.0,0.522460,0.527854
88,1,25323.0,25323.0,0.0,0.0,0.20,0.21,0.20,0.2,653.0,...,368.830200,352.0,53.490002,55.590000,60.000000,58.180000,66.790001,1.0,0.538629,0.530171
89,1,17441.0,17441.0,0.0,0.0,0.20,0.21,0.20,0.2,571.0,...,368.528290,352.0,60.240002,56.160000,54.520000,56.419998,66.849998,1.0,0.515083,0.527590
90,1,11743.0,11743.0,0.0,0.0,0.20,0.21,0.19,0.2,421.0,...,367.166656,352.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.535405,0.528211
91,1,0.0,0.0,6473.0,6473.0,0.20,0.21,0.20,0.2,171.0,...,369.358490,353.0,59.380001,57.130001,50.150002,57.169998,64.010002,1.0,0.544584,0.531765
92,1,0.0,0.0,1561.0,1561.0,0.20,0.21,0.20,0.2,67.0,...,368.358490,353.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.543957,0.537814
93,1,2947.0,2947.0,0.0,0.0,0.20,0.21,0.19,0.2,123.0,...,367.716980,352.0,48.279999,51.250000,55.430000,56.950001,67.440002,1.0,0.519451,0.527599


In [180]:
T05_final_regression_model = finalize_model(T05_model_gbr)

In [181]:
T05_prediction = predict_model(T05_final_regression_model, data=T05_data)
#prediction.drop(['Y_Quality'], axis = 1, inplace = True)

In [182]:
T05_y_class = pd.DataFrame(T05_class, columns = ['Y_Class'])
T05_idx = T05_y_class.index
T05_y_class = T05_y_class.reset_index()
T05_prediction = pd.concat([T05_prediction, T05_y_class], axis = 1)
T05_prediction = T05_prediction.set_index('index')
T05_prediction.drop(['Y_Class'], axis = 1, inplace = True)

In [183]:
T05_prediction['Y_Class'] = 22

T05_idx = T05_prediction.index
for i in range (len(T05_prediction)) :
    if (T05_prediction.iloc[i]['prediction_label'] <= (0.525046 + 0.525086)/2) :
        T05_prediction.loc[T05_idx[i], 'Y_Class'] = 0
    elif ((0.525046 + 0.525086)/2 < T05_prediction.iloc[i]['prediction_label']) and (T05_prediction.iloc[i]['prediction_label'] < (0.534843 + 0.535405)/2) :
        T05_prediction.loc[T05_idx[i], 'Y_Class'] = 1
    elif ((0.534843 + 0.535405)/2 <= T05_prediction.iloc[i]['prediction_label']) :
        T05_prediction.loc[T05_idx[i], 'Y_Class'] = 2
T05_prediction

,PRODUCT_CODE,X_128,X_129,X_130,X_131,X_132,X_133,X_134,X_135,X_136,...,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,prediction_label,Y_Class
index,,,,,,,,,,,,,,,,,,,,,
0,1,7813.0,7813.0,0.0,0.0,0.19,0.20,0.19,0.2,228.0,...,368.296295,353.0,39.340000,40.889999,32.560001,34.090000,77.769997,0.0,0.533353,1
1,1,0.0,0.0,19854.0,19854.0,0.20,0.21,0.20,0.2,413.0,...,367.735840,353.0,38.889999,42.820000,43.919998,35.340000,72.550003,0.0,0.541819,2
2,1,7815.0,7815.0,0.0,0.0,0.19,0.20,0.19,0.2,228.0,...,367.320740,353.0,39.189999,36.650002,42.470001,36.529999,78.349998,0.0,0.531420,1
3,1,0.0,0.0,19856.0,19856.0,0.20,0.21,0.20,0.2,414.0,...,369.188690,353.0,37.740002,39.169998,52.169998,30.580000,71.779999,0.0,0.537425,2
4,1,7817.0,7817.0,0.0,0.0,0.19,0.20,0.18,0.2,228.0,...,367.351837,352.0,38.700001,41.889999,46.930000,33.090000,76.970001,0.0,0.531584,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581,1,11864.0,11864.0,0.0,0.0,0.19,0.20,0.19,0.2,230.0,...,367.333344,352.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.523439,0
582,1,11898.0,11898.0,0.0,0.0,0.18,0.20,0.18,0.2,230.0,...,367.037048,352.0,50.880001,53.230000,52.439999,56.279999,66.830002,1.0,0.522299,0
583,1,11920.0,11920.0,0.0,0.0,0.19,0.20,0.19,0.2,230.0,...,371.943390,353.0,51.709999,59.639999,54.610001,57.049999,63.180000,1.0,0.522449,0


In [184]:
T05_prediction['Y_Class'].unique()

array([1, 2, 0], dtype=int64)

# T01

In [185]:
T01_class = T01['Y_Class']
T01_target = T01['Y_Quality']
T01_data = T01.drop(['Y_Class', 'Y_Quality'], axis = 1)
T01_data

,PRODUCT_CODE,X_246,X_247,X_248,X_249,X_250,X_251,X_252,X_253,X_254,...,X_2856,X_2857,X_2858,X_2859,X_2860,X_2861,X_2862,X_2863,X_2864,X_2865
27,1,1.0,87.0,144.0,45.0,12.0,0.0,45.0,12.0,51.0,...,177.1,186.2,169.907143,156.5,188.4,175.225000,159.9,446.0,430.442308,413.0
28,1,1.0,89.0,0.0,45.0,11.0,90.0,45.0,11.0,42.0,...,170.5,186.8,182.758621,175.7,182.0,174.727586,160.5,446.0,429.716981,413.0
31,1,4.0,93.0,150.0,45.0,12.0,0.0,45.0,12.0,51.0,...,170.8,207.8,191.775862,166.0,195.1,188.696552,181.7,446.0,429.245283,413.0
32,1,1.0,89.0,152.0,45.0,12.0,0.0,45.0,12.0,51.0,...,174.5,190.7,181.686207,172.7,180.0,168.775862,154.1,446.0,429.981132,415.0
33,1,1.0,95.0,0.0,45.0,11.0,98.0,45.0,11.0,42.0,...,173.2,195.0,184.365517,171.4,175.7,168.258621,158.3,446.0,429.301887,413.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,1,1.0,90.0,142.0,45.0,11.0,0.0,45.0,12.0,51.0,...,189.0,201.3,190.407143,181.1,192.6,185.207143,173.2,466.0,446.038461,427.0
553,1,2.0,90.0,0.0,45.0,11.0,122.0,45.0,11.0,42.0,...,215.1,222.4,211.362069,195.7,201.2,191.286207,181.7,466.0,444.924528,424.0
554,1,1.0,89.0,0.0,45.0,11.0,122.0,45.0,11.0,42.0,...,214.1,218.3,211.517857,205.1,193.9,185.071429,177.7,465.0,441.903846,423.0
584,1,1.0,89.0,0.0,45.0,11.0,11.0,45.0,11.0,42.0,...,203.1,208.5,201.160714,193.0,207.1,195.682143,184.9,466.0,448.634615,432.0


In [186]:
#13
setup_rgs = setup(data = T01_data, target=T01_target, session_id = 414)
top5 = compare_models(sort='MSE', n_select=5)

,Description,Value
0,Session id,414
1,Target,Y_Quality
2,Target type,Regression
3,Original data shape,"(129, 896)"
4,Transformed data shape,"(129, 896)"
5,Transformed train set shape,"(90, 896)"
6,Transformed test set shape,"(39, 896)"
7,Numeric features,895
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,0.0043,0.0000,0.0056,0.4950,0.0036,0.0082,0.0390
ada,AdaBoost Regressor,0.0041,0.0000,0.0051,0.5630,0.0034,0.0077,0.0350
et,Extra Trees Regressor,0.0039,0.0000,0.0048,0.5871,0.0031,0.0073,0.0400
catboost,CatBoost Regressor,0.0044,0.0000,0.0057,0.5138,0.0037,0.0082,0.0450
gbr,Gradient Boosting Regressor,0.0037,0.0000,0.0048,0.6241,0.0031,0.0070,0.0360
xgboost,Extreme Gradient Boosting,0.0039,0.0000,0.0049,0.5912,0.0032,0.0074,0.0310
en,Elastic Net,0.0058,0.0001,0.0075,0.2006,0.0049,0.0109,0.0350
lightgbm,Light Gradient Boosting Machine,0.0057,0.0001,0.0072,0.2337,0.0047,0.0107,0.0290
lasso,Lasso Regression,0.0058,0.0001,0.0075,0.2165,0.0049,0.0109,0.0350
ridge,Ridge Regression,0.0061,0.0001,0.0080,0.0786,0.0052,0.0114,0.0300


Processing:   0%|          | 0/89 [00:00<?, ?it/s]

In [187]:
T01_total_models = []

T01_model_lightgbm = create_model('lightgbm', fold = 5)
T01_total_models.append(T01_model_lightgbm)

T01_model_rf = create_model('rf', fold = 5)
T01_total_models.append(T01_model_rf)

T01_model_gbr = create_model('gbr', fold = 5)
T01_total_models.append(T01_model_gbr)

T01_model_et = create_model('et', fold = 5)
T01_total_models.append(T01_model_et)

T01_model_catboost = create_model('catboost', fold = 5)
T01_total_models.append(T01_model_catboost)

T01_model_xgb = create_model('xgboost', fold = 5)
T01_total_models.append(T01_model_xgb)

T01_model_ada = create_model('ada', fold = 5)
T01_total_models.append(T01_model_ada)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0052,0.0001,0.0073,-0.0045,0.0047,0.0096
1,0.0050,0.0000,0.0070,0.4870,0.0046,0.0095
2,0.0067,0.0001,0.0082,-0.7700,0.0054,0.0126
3,0.0091,0.0001,0.0121,0.2170,0.0079,0.0171
4,0.0053,0.0000,0.0063,0.1515,0.0041,0.0100
Mean,0.0063,0.0001,0.0082,0.0162,0.0053,0.0118
Std,0.0016,0.0000,0.0020,0.4240,0.0013,0.0029


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0039,0.0000,0.0048,0.5716,0.0031,0.0073
1,0.0045,0.0000,0.0066,0.5439,0.0043,0.0085
2,0.0044,0.0000,0.0057,0.1635,0.0037,0.0083
3,0.0075,0.0001,0.0097,0.4927,0.0063,0.0141
4,0.0031,0.0000,0.0036,0.7261,0.0023,0.0058
Mean,0.0047,0.0000,0.0061,0.4996,0.0039,0.0088
Std,0.0015,0.0000,0.0021,0.1852,0.0014,0.0028


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0029,0.0000,0.0039,0.7191,0.0025,0.0054
1,0.0040,0.0000,0.0046,0.7733,0.0030,0.0075
2,0.0048,0.0000,0.0060,0.0505,0.0039,0.0091
3,0.0059,0.0001,0.0074,0.7054,0.0048,0.0112
4,0.0024,0.0000,0.0030,0.8069,0.0020,0.0046
Mean,0.0040,0.0000,0.0050,0.6110,0.0033,0.0076
Std,0.0013,0.0000,0.0016,0.2827,0.0010,0.0024


...

Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0042,0.0000,0.0053,0.4659,0.0035,0.0079
1,0.0054,0.0000,0.0070,0.4896,0.0045,0.0102
2,0.0055,0.0000,0.0070,-0.2734,0.0046,0.0103
3,0.0055,0.0000,0.0068,0.7486,0.0045,0.0104
4,0.0022,0.0000,0.0031,0.7967,0.0020,0.0041
Mean,0.0046,0.0000,0.0058,0.4455,0.0038,0.0086
Std,0.0013,0.0000,0.0015,0.3832,0.0010,0.0024


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0031,0.0000,0.0042,0.6664,0.0027,0.0057
1,0.0050,0.0000,0.0065,0.5525,0.0042,0.0094
2,0.0047,0.0000,0.0058,0.1338,0.0038,0.0088
3,0.0074,0.0001,0.0103,0.4253,0.0067,0.0139
4,0.0030,0.0000,0.0035,0.7416,0.0023,0.0056
Mean,0.0047,0.0000,0.0061,0.5039,0.0039,0.0087
Std,0.0016,0.0000,0.0024,0.2137,0.0016,0.0030


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [188]:
T01_model_lightgbm = tune_model(T01_model_lightgbm, fold=5, choose_better = True)

T01_model_rf = tune_model(T01_model_rf, fold=5, choose_better = True)

T01_model_gbr = tune_model(T01_model_gbr, fold=5, choose_better = True)

T01_model_et = tune_model(T01_model_et, fold=5, choose_better = True)

T01_model_catboost = tune_model(T01_model_catboost, fold=5, choose_better = True)

T01_model_xgb = tune_model(T01_model_xgb, fold=5, choose_better = True)

T01_model_ada = tune_model(T01_model_ada, fold=5, choose_better = True)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0048,0.0001,0.0075,-0.0552,0.0049,0.0088
1,0.0072,0.0001,0.0098,-0.0033,0.0064,0.0136
2,0.0047,0.0000,0.0062,-0.0098,0.0041,0.0088
3,0.0097,0.0002,0.0137,-0.0057,0.0089,0.0183
4,0.0053,0.0000,0.0068,-0.0050,0.0045,0.0099
Mean,0.0063,0.0001,0.0088,-0.0158,0.0057,0.0119
Std,0.0019,0.0001,0.0027,0.0198,0.0018,0.0037


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0047,0.0001,0.0075,-0.0654,0.0049,0.0087
1,0.0068,0.0001,0.0091,0.1307,0.0059,0.0127
2,0.0045,0.0000,0.0060,0.0490,0.0039,0.0084
3,0.0091,0.0002,0.0131,0.0789,0.0085,0.0172
4,0.0047,0.0000,0.0060,0.2281,0.0039,0.0087
Mean,0.0059,0.0001,0.0083,0.0843,0.0054,0.0111
Std,0.0018,0.0001,0.0026,0.0965,0.0017,0.0034


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

...

Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 5 folds for each of 10 candidates, totalling 50 fits


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0035,0.0000,0.0044,0.6369,0.0029,0.0066
1,0.0049,0.0000,0.0065,0.5597,0.0042,0.0092
2,0.0045,0.0000,0.0057,0.1554,0.0037,0.0084
3,0.0073,0.0001,0.0101,0.4488,0.0066,0.0137
4,0.0030,0.0000,0.0034,0.7486,0.0022,0.0055
Mean,0.0046,0.0000,0.0060,0.5099,0.0039,0.0087
Std,0.0015,0.0000,0.0023,0.2025,0.0015,0.0028


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [189]:
T01_tuned_models = [T01_model_lightgbm, T01_model_rf, T01_model_gbr, T01_model_et, T01_model_catboost, T01_model_xgb, T01_model_ada]
T01_tuned_models

[LGBMRegressor(random_state=414),
 RandomForestRegressor(n_jobs=-1, random_state=414),
 GradientBoostingRegressor(random_state=414),
 ExtraTreesRegressor(n_jobs=-1, random_state=414),
 XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.5,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.2, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=7, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=90, n_jobs=-1,
              num_parallel_tree=1, predictor='auto', random_state=414,
              reg_alpha=0.01, reg_lambda=0.05, ...),
 AdaBoostRegressor(learning_rate=0.05, n_estimators=290, random_state=414)]

In [190]:
for model in T01_tuned_models:
    display(predict_model(model))

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,0.0053,0.0000,0.0067,0.4065,0.0044,0.0099


,PRODUCT_CODE,X_246,X_247,X_248,X_249,X_250,X_251,X_252,X_253,X_254,...,X_2858,X_2859,X_2860,X_2861,X_2862,X_2863,X_2864,X_2865,Y_Quality,prediction_label
90,1,1.0,95.0,0.0,45.0,11.0,148.0,45.0,11.0,42.0,...,171.910339,169.100006,198.899994,188.513794,181.000000,456.0,439.415100,422.0,0.534448,0.531623
91,1,2.0,89.0,0.0,45.0,11.0,135.0,45.0,11.0,42.0,...,169.139282,164.100006,160.800003,156.482147,142.000000,466.0,450.000000,432.0,0.534376,0.536807
92,1,2.0,91.0,170.0,45.0,11.0,0.0,45.0,12.0,51.0,...,187.639282,160.500000,194.100006,180.189285,165.500000,446.0,430.942322,413.0,0.547184,0.547613
93,1,2.0,88.0,0.0,45.0,11.0,85.0,45.0,11.0,42.0,...,191.772415,179.000000,181.399994,165.093109,160.100006,456.0,439.735840,422.0,0.533960,0.541074
94,1,1.0,88.0,92.0,45.0,12.0,0.0,45.0,12.0,51.0,...,196.762070,188.399994,188.300003,177.379303,166.600006,456.0,439.396240,415.0,0.528087,0.534987
95,1,1.0,95.0,0.0,45.0,11.0,22.0,45.0,11.0,42.0,...,181.217850,161.100006,185.100006,167.035721,157.300003,466.0,448.442322,433.0,0.528190,0.531168
96,1,1.0,90.0,0.0,45.0,11.0,46.0,45.0,11.0,42.0,...,190.350006,175.699997,177.600006,168.635712,160.699997,465.0,445.735840,423.0,0.527486,0.533454
97,1,1.0,95.0,0.0,45.0,11.0,28.0,45.0,11.0,42.0,...,186.274994,165.000000,174.699997,166.457138,150.399994,456.0,438.615387,422.0,0.530048,0.536075
98,1,2.0,88.0,0.0,45.0,11.0,42.0,45.0,11.0,42.0,...,194.464279,185.300003,173.100006,162.460709,154.000000,456.0,440.288452,424.0,0.530514,0.532849
99,1,1.0,90.0,142.0,45.0,11.0,0.0,45.0,12.0,51.0,...,190.407150,181.100006,192.600006,185.207138,173.199997,466.0,446.038452,427.0,0.560317,0.542315


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Random Forest Regressor,0.0039,0.0000,0.0050,0.6771,0.0032,0.0074


,PRODUCT_CODE,X_246,X_247,X_248,X_249,X_250,X_251,X_252,X_253,X_254,...,X_2858,X_2859,X_2860,X_2861,X_2862,X_2863,X_2864,X_2865,Y_Quality,prediction_label
90,1,1.0,95.0,0.0,45.0,11.0,148.0,45.0,11.0,42.0,...,171.910339,169.100006,198.899994,188.513794,181.000000,456.0,439.415100,422.0,0.534448,0.533404
91,1,2.0,89.0,0.0,45.0,11.0,135.0,45.0,11.0,42.0,...,169.139282,164.100006,160.800003,156.482147,142.000000,466.0,450.000000,432.0,0.534376,0.533651
92,1,2.0,91.0,170.0,45.0,11.0,0.0,45.0,12.0,51.0,...,187.639282,160.500000,194.100006,180.189285,165.500000,446.0,430.942322,413.0,0.547184,0.543830
93,1,2.0,88.0,0.0,45.0,11.0,85.0,45.0,11.0,42.0,...,191.772415,179.000000,181.399994,165.093109,160.100006,456.0,439.735840,422.0,0.533960,0.541807
94,1,1.0,88.0,92.0,45.0,12.0,0.0,45.0,12.0,51.0,...,196.762070,188.399994,188.300003,177.379303,166.600006,456.0,439.396240,415.0,0.528087,0.532111
95,1,1.0,95.0,0.0,45.0,11.0,22.0,45.0,11.0,42.0,...,181.217850,161.100006,185.100006,167.035721,157.300003,466.0,448.442322,433.0,0.528190,0.530271
96,1,1.0,90.0,0.0,45.0,11.0,46.0,45.0,11.0,42.0,...,190.350006,175.699997,177.600006,168.635712,160.699997,465.0,445.735840,423.0,0.527486,0.530185
97,1,1.0,95.0,0.0,45.0,11.0,28.0,45.0,11.0,42.0,...,186.274994,165.000000,174.699997,166.457138,150.399994,456.0,438.615387,422.0,0.530048,0.530845
98,1,2.0,88.0,0.0,45.0,11.0,42.0,45.0,11.0,42.0,...,194.464279,185.300003,173.100006,162.460709,154.000000,456.0,440.288452,424.0,0.530514,0.529606
99,1,1.0,90.0,142.0,45.0,11.0,0.0,45.0,12.0,51.0,...,190.407150,181.100006,192.600006,185.207138,173.199997,466.0,446.038452,427.0,0.560317,0.549129


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Gradient Boosting Regressor,0.0039,0.0000,0.0048,0.6977,0.0031,0.0072


...

,PRODUCT_CODE,X_246,X_247,X_248,X_249,X_250,X_251,X_252,X_253,X_254,...,X_2858,X_2859,X_2860,X_2861,X_2862,X_2863,X_2864,X_2865,Y_Quality,prediction_label
90,1,1.0,95.0,0.0,45.0,11.0,148.0,45.0,11.0,42.0,...,171.910339,169.100006,198.899994,188.513794,181.000000,456.0,439.415100,422.0,0.534448,0.535083
91,1,2.0,89.0,0.0,45.0,11.0,135.0,45.0,11.0,42.0,...,169.139282,164.100006,160.800003,156.482147,142.000000,466.0,450.000000,432.0,0.534376,0.533323
92,1,2.0,91.0,170.0,45.0,11.0,0.0,45.0,12.0,51.0,...,187.639282,160.500000,194.100006,180.189285,165.500000,446.0,430.942322,413.0,0.547184,0.547553
93,1,2.0,88.0,0.0,45.0,11.0,85.0,45.0,11.0,42.0,...,191.772415,179.000000,181.399994,165.093109,160.100006,456.0,439.735840,422.0,0.533960,0.537872
94,1,1.0,88.0,92.0,45.0,12.0,0.0,45.0,12.0,51.0,...,196.762070,188.399994,188.300003,177.379303,166.600006,456.0,439.396240,415.0,0.528087,0.531129
95,1,1.0,95.0,0.0,45.0,11.0,22.0,45.0,11.0,42.0,...,181.217850,161.100006,185.100006,167.035721,157.300003,466.0,448.442322,433.0,0.528190,0.530766
96,1,1.0,90.0,0.0,45.0,11.0,46.0,45.0,11.0,42.0,...,190.350006,175.699997,177.600006,168.635712,160.699997,465.0,445.735840,423.0,0.527486,0.531626
97,1,1.0,95.0,0.0,45.0,11.0,28.0,45.0,11.0,42.0,...,186.274994,165.000000,174.699997,166.457138,150.399994,456.0,438.615387,422.0,0.530048,0.531634
98,1,2.0,88.0,0.0,45.0,11.0,42.0,45.0,11.0,42.0,...,194.464279,185.300003,173.100006,162.460709,154.000000,456.0,440.288452,424.0,0.530514,0.528038
99,1,1.0,90.0,142.0,45.0,11.0,0.0,45.0,12.0,51.0,...,190.407150,181.100006,192.600006,185.207138,173.199997,466.0,446.038452,427.0,0.560317,0.542481


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extreme Gradient Boosting,0.0040,0.0000,0.0051,0.6602,0.0033,0.0076


,PRODUCT_CODE,X_246,X_247,X_248,X_249,X_250,X_251,X_252,X_253,X_254,...,X_2858,X_2859,X_2860,X_2861,X_2862,X_2863,X_2864,X_2865,Y_Quality,prediction_label
90,1,1.0,95.0,0.0,45.0,11.0,148.0,45.0,11.0,42.0,...,171.910339,169.100006,198.899994,188.513794,181.000000,456.0,439.415100,422.0,0.534448,0.532178
91,1,2.0,89.0,0.0,45.0,11.0,135.0,45.0,11.0,42.0,...,169.139282,164.100006,160.800003,156.482147,142.000000,466.0,450.000000,432.0,0.534376,0.535314
92,1,2.0,91.0,170.0,45.0,11.0,0.0,45.0,12.0,51.0,...,187.639282,160.500000,194.100006,180.189285,165.500000,446.0,430.942322,413.0,0.547184,0.547710
93,1,2.0,88.0,0.0,45.0,11.0,85.0,45.0,11.0,42.0,...,191.772415,179.000000,181.399994,165.093109,160.100006,456.0,439.735840,422.0,0.533960,0.538015
94,1,1.0,88.0,92.0,45.0,12.0,0.0,45.0,12.0,51.0,...,196.762070,188.399994,188.300003,177.379303,166.600006,456.0,439.396240,415.0,0.528087,0.529467
95,1,1.0,95.0,0.0,45.0,11.0,22.0,45.0,11.0,42.0,...,181.217850,161.100006,185.100006,167.035721,157.300003,466.0,448.442322,433.0,0.528190,0.531491
96,1,1.0,90.0,0.0,45.0,11.0,46.0,45.0,11.0,42.0,...,190.350006,175.699997,177.600006,168.635712,160.699997,465.0,445.735840,423.0,0.527486,0.528501
97,1,1.0,95.0,0.0,45.0,11.0,28.0,45.0,11.0,42.0,...,186.274994,165.000000,174.699997,166.457138,150.399994,456.0,438.615387,422.0,0.530048,0.532667
98,1,2.0,88.0,0.0,45.0,11.0,42.0,45.0,11.0,42.0,...,194.464279,185.300003,173.100006,162.460709,154.000000,456.0,440.288452,424.0,0.530514,0.530235
99,1,1.0,90.0,142.0,45.0,11.0,0.0,45.0,12.0,51.0,...,190.407150,181.100006,192.600006,185.207138,173.199997,466.0,446.038452,427.0,0.560317,0.549682


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,AdaBoost Regressor,0.0037,0.0000,0.0045,0.7322,0.0029,0.0070


,PRODUCT_CODE,X_246,X_247,X_248,X_249,X_250,X_251,X_252,X_253,X_254,...,X_2858,X_2859,X_2860,X_2861,X_2862,X_2863,X_2864,X_2865,Y_Quality,prediction_label
90,1,1.0,95.0,0.0,45.0,11.0,148.0,45.0,11.0,42.0,...,171.910339,169.100006,198.899994,188.513794,181.000000,456.0,439.415100,422.0,0.534448,0.532723
91,1,2.0,89.0,0.0,45.0,11.0,135.0,45.0,11.0,42.0,...,169.139282,164.100006,160.800003,156.482147,142.000000,466.0,450.000000,432.0,0.534376,0.532451
92,1,2.0,91.0,170.0,45.0,11.0,0.0,45.0,12.0,51.0,...,187.639282,160.500000,194.100006,180.189285,165.500000,446.0,430.942322,413.0,0.547184,0.550958
93,1,2.0,88.0,0.0,45.0,11.0,85.0,45.0,11.0,42.0,...,191.772415,179.000000,181.399994,165.093109,160.100006,456.0,439.735840,422.0,0.533960,0.537108
94,1,1.0,88.0,92.0,45.0,12.0,0.0,45.0,12.0,51.0,...,196.762070,188.399994,188.300003,177.379303,166.600006,456.0,439.396240,415.0,0.528087,0.531437
95,1,1.0,95.0,0.0,45.0,11.0,22.0,45.0,11.0,42.0,...,181.217850,161.100006,185.100006,167.035721,157.300003,466.0,448.442322,433.0,0.528190,0.530146
96,1,1.0,90.0,0.0,45.0,11.0,46.0,45.0,11.0,42.0,...,190.350006,175.699997,177.600006,168.635712,160.699997,465.0,445.735840,423.0,0.527486,0.530399
97,1,1.0,95.0,0.0,45.0,11.0,28.0,45.0,11.0,42.0,...,186.274994,165.000000,174.699997,166.457138,150.399994,456.0,438.615387,422.0,0.530048,0.530654
98,1,2.0,88.0,0.0,45.0,11.0,42.0,45.0,11.0,42.0,...,194.464279,185.300003,173.100006,162.460709,154.000000,456.0,440.288452,424.0,0.530514,0.530152
99,1,1.0,90.0,142.0,45.0,11.0,0.0,45.0,12.0,51.0,...,190.407150,181.100006,192.600006,185.207138,173.199997,466.0,446.038452,427.0,0.560317,0.551058


In [191]:
T01_final_regression_model = finalize_model(T01_model_et)

In [192]:
T01_prediction = predict_model(T01_final_regression_model, data=T01_data)
#prediction.drop(['Y_Quality'], axis = 1, inplace = True)

In [193]:
T01_y_class = pd.DataFrame(T01_class, columns = ['Y_Class'])
T01_idx = T01_y_class.index
T01_y_class = T01_y_class.reset_index()
T01_prediction = pd.concat([T01_prediction, T01_y_class], axis = 1)
T01_prediction = T01_prediction.set_index('index')
T01_prediction.drop(['Y_Class'], axis = 1, inplace = True)

In [196]:
T01_prediction['Y_Class'] = 22

T01_idx = T01_prediction.index
for i in range (len(T01_prediction)) :
    if (T01_prediction.iloc[i]['prediction_label'] <= (0.524808 + 0.525221)/2) :
        T01_prediction.loc[T01_idx[i], 'Y_Class'] = 0
    elif ((0.524808 + 0.525221)/2 < T01_prediction.iloc[i]['prediction_label']) and (T01_prediction.iloc[i]['prediction_label'] < (0.534737 + 0.535279)/2) :
        T01_prediction.loc[T01_idx[i], 'Y_Class'] = 1
    elif ((0.534737 + 0.535279)/2 <= T01_prediction.iloc[i]['prediction_label']) :
        T01_prediction.loc[T01_idx[i], 'Y_Class'] = 2
T01_prediction

,PRODUCT_CODE,X_246,X_247,X_248,X_249,X_250,X_251,X_252,X_253,X_254,...,X_2858,X_2859,X_2860,X_2861,X_2862,X_2863,X_2864,X_2865,prediction_label,Y_Class
index,,,,,,,,,,,,,,,,,,,,,
27,1,1.0,87.0,144.0,45.0,12.0,0.0,45.0,12.0,51.0,...,169.907150,156.500000,188.399994,175.225006,159.899994,446.0,430.442322,413.0,0.540002,2
28,1,1.0,89.0,0.0,45.0,11.0,90.0,45.0,11.0,42.0,...,182.758621,175.699997,182.000000,174.727585,160.500000,446.0,429.716980,413.0,0.521249,0
31,1,4.0,93.0,150.0,45.0,12.0,0.0,45.0,12.0,51.0,...,191.775864,166.000000,195.100006,188.696548,181.699997,446.0,429.245270,413.0,0.536206,2
32,1,1.0,89.0,152.0,45.0,12.0,0.0,45.0,12.0,51.0,...,181.686203,172.699997,180.000000,168.775864,154.100006,446.0,429.981140,415.0,0.548592,2
33,1,1.0,95.0,0.0,45.0,11.0,98.0,45.0,11.0,42.0,...,184.365524,171.399994,175.699997,168.258621,158.300003,446.0,429.301880,413.0,0.526656,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,1,1.0,90.0,142.0,45.0,11.0,0.0,45.0,12.0,51.0,...,190.407150,181.100006,192.600006,185.207138,173.199997,466.0,446.038452,427.0,0.560317,2
553,1,2.0,90.0,0.0,45.0,11.0,122.0,45.0,11.0,42.0,...,211.362076,195.699997,201.199997,191.286209,181.699997,466.0,444.924530,424.0,0.524686,0
554,1,1.0,89.0,0.0,45.0,11.0,122.0,45.0,11.0,42.0,...,211.517853,205.100006,193.899994,185.071426,177.699997,465.0,441.903839,423.0,0.531106,1


In [197]:
T01_prediction['Y_Class'].unique()

array([2, 0, 1], dtype=int64)

# T10

In [198]:
T10_class = T10['Y_Class']
T10_target = T10['Y_Quality']
T10_data = T10.drop(['Y_Class', 'Y_Quality'], axis = 1)
T10_data

,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,...,X_924,X_925,X_926,X_927,X_928,X_929,X_930,X_931,X_932,X_933
22,2,2.0,102.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,...,172.5,162.8,157.235484,150.9,326.0,309.290909,290.0,13.7,13.443333,13.2
23,2,2.0,102.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
25,2,2.0,97.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,...,177.8,157.1,154.074194,150.9,327.0,310.854546,298.0,13.7,13.454839,13.2
26,2,2.0,95.0,0.0,45.0,10.0,0.0,54.0,10.0,52.0,...,166.9,158.5,151.096774,146.5,327.0,313.618182,299.0,13.7,13.461290,13.3
29,2,2.0,100.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,...,177.9,161.3,156.038710,149.1,327.0,312.109091,298.0,13.7,13.422581,13.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
591,2,1.0,87.0,0.0,45.0,10.0,0.0,50.0,10.0,52.0,...,177.0,159.6,154.777419,147.7,326.0,313.472727,298.0,13.6,13.400000,13.2
592,2,2.0,98.0,0.0,45.0,10.0,0.0,45.0,10.0,31.0,...,171.5,156.6,152.535484,147.6,326.0,312.472727,298.0,13.6,13.361290,13.2
593,2,2.0,95.0,0.0,45.0,10.0,0.0,50.0,10.0,52.0,...,172.8,161.2,155.438710,149.0,327.0,313.363636,298.0,13.6,13.351613,13.2
596,3,40.0,94.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,...,189.2,179.5,173.190323,168.6,321.0,303.036364,289.0,13.6,13.406667,13.2


In [199]:
#13
setup_rgs = setup(data = T10_data, target=T10_target, session_id = 414)
top5 = compare_models(sort='MSE', n_select=5)

,Description,Value
0,Session id,414
1,Target,Y_Quality
2,Target type,Regression
3,Original data shape,"(349, 679)"
4,Transformed data shape,"(349, 679)"
5,Transformed train set shape,"(244, 679)"
6,Transformed test set shape,"(105, 679)"
7,Numeric features,678
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lasso,Lasso Regression,0.0030,0.0000,0.0044,-0.0653,0.0029,0.0057,0.0270
catboost,CatBoost Regressor,0.0028,0.0000,0.0038,0.1636,0.0025,0.0053,0.0370
llar,Lasso Least Angle Regression,0.0030,0.0000,0.0044,-0.0289,0.0028,0.0056,0.0270
xgboost,Extreme Gradient Boosting,0.0033,0.0000,0.0045,-0.2474,0.0030,0.0063,0.0290
br,Bayesian Ridge,0.0038,0.0000,0.0051,-0.6482,0.0033,0.0071,0.0330
ridge,Ridge Regression,0.0034,0.0000,0.0048,-0.3069,0.0031,0.0064,0.0260
omp,Orthogonal Matching Pursuit,0.0036,0.0000,0.0052,-0.5325,0.0034,0.0069,0.0290
en,Elastic Net,0.0030,0.0000,0.0044,-0.0762,0.0029,0.0057,0.0270
knn,K Neighbors Regressor,0.0034,0.0000,0.0046,-0.2505,0.0030,0.0063,0.0300
dt,Decision Tree Regressor,0.0043,0.0000,0.0060,-1.2017,0.0039,0.0081,0.0350


Processing:   0%|          | 0/89 [00:00<?, ?it/s]

In [200]:
T10_total_models = []

T10_model_rf = create_model('rf', fold = 5)
T10_total_models.append(T10_model_rf)

T10_model_gbr = create_model('gbr', fold = 5)
T10_total_models.append(T10_model_gbr)

T10_model_et = create_model('et', fold = 5)
T10_total_models.append(T10_model_et)

T10_model_catboost = create_model('catboost', fold = 5)
T10_total_models.append(T10_model_catboost)

T10_model_xgb = create_model('xgboost', fold = 5)
T10_total_models.append(T10_model_xgb)

T10_model_ada = create_model('ada', fold = 5)
T10_total_models.append(T10_model_ada)

T10_model_lightgbm = create_model('lightgbm', fold = 5)
T10_total_models.append(T10_model_lightgbm)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0031,0.0000,0.0039,-0.0717,0.0026,0.0059
1,0.0026,0.0000,0.0035,-0.2830,0.0023,0.0049
2,0.0033,0.0000,0.0047,0.2514,0.0031,0.0062
3,0.0035,0.0000,0.0047,0.2104,0.0031,0.0065
4,0.0028,0.0000,0.0041,0.1573,0.0027,0.0052
Mean,0.0030,0.0000,0.0042,0.0529,0.0027,0.0057
Std,0.0003,0.0000,0.0005,0.2017,0.0003,0.0006


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0028,0.0000,0.0035,0.1480,0.0023,0.0054
1,0.0024,0.0000,0.0031,-0.0305,0.0020,0.0045
2,0.0031,0.0000,0.0042,0.4075,0.0027,0.0060
3,0.0034,0.0000,0.0046,0.2455,0.0030,0.0064
4,0.0028,0.0000,0.0041,0.1448,0.0027,0.0052
Mean,0.0029,0.0000,0.0039,0.1831,0.0026,0.0055
Std,0.0003,0.0000,0.0005,0.1432,0.0003,0.0007


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0030,0.0000,0.0039,-0.0743,0.0026,0.0057
1,0.0025,0.0000,0.0032,-0.1093,0.0021,0.0047
2,0.0031,0.0000,0.0044,0.3477,0.0029,0.0059
3,0.0029,0.0000,0.0043,0.3599,0.0028,0.0055
4,0.0028,0.0000,0.0041,0.1366,0.0027,0.0053
Mean,0.0029,0.0000,0.0040,0.1321,0.0026,0.0054
Std,0.0002,0.0000,0.0004,0.1996,0.0003,0.0004


...

Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0030,0.0000,0.0038,0.0076,0.0025,0.0056
1,0.0023,0.0000,0.0031,0.0149,0.0020,0.0044
2,0.0035,0.0000,0.0052,0.0708,0.0034,0.0066
3,0.0033,0.0000,0.0050,0.1058,0.0033,0.0062
4,0.0026,0.0000,0.0040,0.1891,0.0026,0.0050
Mean,0.0029,0.0000,0.0042,0.0776,0.0028,0.0055
Std,0.0004,0.0000,0.0008,0.0665,0.0005,0.0008


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0024,0.0000,0.0030,0.3648,0.0020,0.0044
1,0.0024,0.0000,0.0033,-0.1405,0.0021,0.0046
2,0.0036,0.0000,0.0052,0.0728,0.0034,0.0068
3,0.0033,0.0000,0.0047,0.2295,0.0031,0.0062
4,0.0029,0.0000,0.0041,0.1498,0.0027,0.0054
Mean,0.0029,0.0000,0.0041,0.1353,0.0027,0.0055
Std,0.0005,0.0000,0.0008,0.1684,0.0005,0.0009


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [201]:
T10_model_rf = tune_model(T10_model_rf, fold=5, choose_better = True)

T10_model_gbr = tune_model(T10_model_gbr, fold=5, choose_better = True)

T10_model_et = tune_model(T10_model_et, fold=5, choose_better = True)

T10_model_catboost = tune_model(T10_model_catboost, fold=5, choose_better = True)

T10_model_xgb = tune_model(T10_model_xgb, fold=5, choose_better = True)

T10_model_ada = tune_model(T10_model_ada, fold=5, choose_better = True)

T10_model_lightgbm = tune_model(T10_model_lightgbm, fold=5, choose_better = True)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0029,0.0000,0.0038,0.0156,0.0025,0.0055
1,0.0023,0.0000,0.0031,0.0064,0.0020,0.0044
2,0.0035,0.0000,0.0055,-0.0124,0.0036,0.0067
3,0.0034,0.0000,0.0053,-0.0046,0.0035,0.0064
4,0.0028,0.0000,0.0045,-0.0103,0.0029,0.0053
Mean,0.0030,0.0000,0.0044,-0.0011,0.0029,0.0056
Std,0.0004,0.0000,0.0009,0.0106,0.0006,0.0008


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0029,0.0000,0.0038,-0.0016,0.0025,0.0055
1,0.0023,0.0000,0.0031,-0.0012,0.0020,0.0044
2,0.0035,0.0000,0.0054,-0.0013,0.0035,0.0066
3,0.0033,0.0000,0.0053,-0.0002,0.0035,0.0063
4,0.0028,0.0000,0.0045,-0.0170,0.0029,0.0053
Mean,0.0030,0.0000,0.0044,-0.0043,0.0029,0.0056
Std,0.0004,0.0000,0.0009,0.0064,0.0006,0.0008


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

...

Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 5 folds for each of 10 candidates, totalling 50 fits


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0029,0.0000,0.0038,-0.0001,0.0025,0.0055
1,0.0023,0.0000,0.0031,-0.0041,0.0020,0.0044
2,0.0035,0.0000,0.0054,-0.0041,0.0036,0.0066
3,0.0033,0.0000,0.0053,-0.0004,0.0035,0.0063
4,0.0028,0.0000,0.0045,-0.0196,0.0029,0.0053
Mean,0.0030,0.0000,0.0044,-0.0057,0.0029,0.0056
Std,0.0004,0.0000,0.0009,0.0072,0.0006,0.0008


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [202]:
T10_tuned_models = [T10_model_rf, T10_model_gbr, T10_model_et, T10_model_catboost, T10_model_xgb, T10_model_ada, T10_model_lightgbm]
T10_tuned_models

[RandomForestRegressor(n_jobs=-1, random_state=414),
 GradientBoostingRegressor(random_state=414),
 ExtraTreesRegressor(n_jobs=-1, random_state=414),
 XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.5,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.2, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=7, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=90, n_jobs=-1,
              num_parallel_tree=1, predictor='auto', random_state=414,
              reg_alpha=0.01, reg_lambda=0.05, ...),
 AdaBoostRegressor(learning_rate=0.05, n_estimators=210, random_state=414),
 LGBMRegressor(random_state=414)]

In [203]:
for model in T10_tuned_models:
    display(predict_model(model))

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Random Forest Regressor,0.0033,0.0000,0.0046,0.2258,0.0030,0.0062


,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,...,X_926,X_927,X_928,X_929,X_930,X_931,X_932,X_933,Y_Quality,prediction_label
244,2,2.0,96.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,...,156.709671,150.699997,317.0,303.836365,288.0,13.7,13.409678,13.2,0.529167,0.530502
245,2,2.0,102.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,...,153.658066,148.100006,327.0,313.636353,298.0,13.6,13.393548,13.2,0.532317,0.530575
246,2,2.0,98.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,...,154.393555,147.500000,326.0,304.436371,289.0,13.6,13.406667,13.2,0.526232,0.530309
247,2,2.0,101.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,...,154.245163,148.100006,326.0,310.509094,289.0,13.6,13.325807,13.1,0.532694,0.529457
248,2,2.0,94.0,0.0,45.0,10.0,0.0,54.0,10.0,52.0,...,152.738708,146.699997,325.0,305.000000,290.0,13.5,13.310000,13.1,0.527637,0.531029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,2,2.0,101.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,...,151.167740,147.100006,326.0,308.309082,289.0,13.6,13.325807,13.2,0.528154,0.531247
345,2,2.0,102.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,...,148.403229,144.100006,327.0,310.363647,298.0,13.6,13.396774,13.2,0.529000,0.529899
346,2,2.0,101.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,...,153.213333,149.100006,327.0,308.370361,289.0,13.7,13.470967,13.3,0.529097,0.528983
347,2,2.0,93.0,0.0,45.0,10.0,0.0,45.0,10.0,31.0,...,148.841934,143.300003,317.0,301.072723,285.0,13.6,13.366667,13.1,0.532446,0.529729


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Gradient Boosting Regressor,0.0031,0.0000,0.0043,0.3141,0.0028,0.0059


,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,...,X_926,X_927,X_928,X_929,X_930,X_931,X_932,X_933,Y_Quality,prediction_label
244,2,2.0,96.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,...,156.709671,150.699997,317.0,303.836365,288.0,13.7,13.409678,13.2,0.529167,0.528185
245,2,2.0,102.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,...,153.658066,148.100006,327.0,313.636353,298.0,13.6,13.393548,13.2,0.532317,0.531393
246,2,2.0,98.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,...,154.393555,147.500000,326.0,304.436371,289.0,13.6,13.406667,13.2,0.526232,0.530762
247,2,2.0,101.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,...,154.245163,148.100006,326.0,310.509094,289.0,13.6,13.325807,13.1,0.532694,0.530538
248,2,2.0,94.0,0.0,45.0,10.0,0.0,54.0,10.0,52.0,...,152.738708,146.699997,325.0,305.000000,290.0,13.5,13.310000,13.1,0.527637,0.530889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,2,2.0,101.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,...,151.167740,147.100006,326.0,308.309082,289.0,13.6,13.325807,13.2,0.528154,0.533632
345,2,2.0,102.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,...,148.403229,144.100006,327.0,310.363647,298.0,13.6,13.396774,13.2,0.529000,0.529565
346,2,2.0,101.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,...,153.213333,149.100006,327.0,308.370361,289.0,13.7,13.470967,13.3,0.529097,0.528973
347,2,2.0,93.0,0.0,45.0,10.0,0.0,45.0,10.0,31.0,...,148.841934,143.300003,317.0,301.072723,285.0,13.6,13.366667,13.1,0.532446,0.530038


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extra Trees Regressor,0.0032,0.0000,0.0044,0.2903,0.0029,0.0060


...

,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,...,X_926,X_927,X_928,X_929,X_930,X_931,X_932,X_933,Y_Quality,prediction_label
244,2,2.0,96.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,...,156.709671,150.699997,317.0,303.836365,288.0,13.7,13.409678,13.2,0.529167,0.529982
245,2,2.0,102.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,...,153.658066,148.100006,327.0,313.636353,298.0,13.6,13.393548,13.2,0.532317,0.529315
246,2,2.0,98.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,...,154.393555,147.500000,326.0,304.436371,289.0,13.6,13.406667,13.2,0.526232,0.533774
247,2,2.0,101.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,...,154.245163,148.100006,326.0,310.509094,289.0,13.6,13.325807,13.1,0.532694,0.530048
248,2,2.0,94.0,0.0,45.0,10.0,0.0,54.0,10.0,52.0,...,152.738708,146.699997,325.0,305.000000,290.0,13.5,13.310000,13.1,0.527637,0.529679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,2,2.0,101.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,...,151.167740,147.100006,326.0,308.309082,289.0,13.6,13.325807,13.2,0.528154,0.534121
345,2,2.0,102.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,...,148.403229,144.100006,327.0,310.363647,298.0,13.6,13.396774,13.2,0.529000,0.531274
346,2,2.0,101.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,...,153.213333,149.100006,327.0,308.370361,289.0,13.7,13.470967,13.3,0.529097,0.531469
347,2,2.0,93.0,0.0,45.0,10.0,0.0,45.0,10.0,31.0,...,148.841934,143.300003,317.0,301.072723,285.0,13.6,13.366667,13.1,0.532446,0.530026


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,AdaBoost Regressor,0.0031,0.0000,0.0046,0.2255,0.0030,0.0060


,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,...,X_926,X_927,X_928,X_929,X_930,X_931,X_932,X_933,Y_Quality,prediction_label
244,2,2.0,96.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,...,156.709671,150.699997,317.0,303.836365,288.0,13.7,13.409678,13.2,0.529167,0.530279
245,2,2.0,102.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,...,153.658066,148.100006,327.0,313.636353,298.0,13.6,13.393548,13.2,0.532317,0.530335
246,2,2.0,98.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,...,154.393555,147.500000,326.0,304.436371,289.0,13.6,13.406667,13.2,0.526232,0.530434
247,2,2.0,101.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,...,154.245163,148.100006,326.0,310.509094,289.0,13.6,13.325807,13.1,0.532694,0.530210
248,2,2.0,94.0,0.0,45.0,10.0,0.0,54.0,10.0,52.0,...,152.738708,146.699997,325.0,305.000000,290.0,13.5,13.310000,13.1,0.527637,0.530433
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,2,2.0,101.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,...,151.167740,147.100006,326.0,308.309082,289.0,13.6,13.325807,13.2,0.528154,0.530351
345,2,2.0,102.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,...,148.403229,144.100006,327.0,310.363647,298.0,13.6,13.396774,13.2,0.529000,0.530237
346,2,2.0,101.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,...,153.213333,149.100006,327.0,308.370361,289.0,13.7,13.470967,13.3,0.529097,0.530283
347,2,2.0,93.0,0.0,45.0,10.0,0.0,45.0,10.0,31.0,...,148.841934,143.300003,317.0,301.072723,285.0,13.6,13.366667,13.1,0.532446,0.530452


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,0.0029,0.0000,0.0042,0.3408,0.0028,0.0055


,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,...,X_926,X_927,X_928,X_929,X_930,X_931,X_932,X_933,Y_Quality,prediction_label
244,2,2.0,96.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,...,156.709671,150.699997,317.0,303.836365,288.0,13.7,13.409678,13.2,0.529167,0.526588
245,2,2.0,102.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,...,153.658066,148.100006,327.0,313.636353,298.0,13.6,13.393548,13.2,0.532317,0.533604
246,2,2.0,98.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,...,154.393555,147.500000,326.0,304.436371,289.0,13.6,13.406667,13.2,0.526232,0.529960
247,2,2.0,101.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,...,154.245163,148.100006,326.0,310.509094,289.0,13.6,13.325807,13.1,0.532694,0.532703
248,2,2.0,94.0,0.0,45.0,10.0,0.0,54.0,10.0,52.0,...,152.738708,146.699997,325.0,305.000000,290.0,13.5,13.310000,13.1,0.527637,0.530480
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,2,2.0,101.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,...,151.167740,147.100006,326.0,308.309082,289.0,13.6,13.325807,13.2,0.528154,0.532983
345,2,2.0,102.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,...,148.403229,144.100006,327.0,310.363647,298.0,13.6,13.396774,13.2,0.529000,0.531742
346,2,2.0,101.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,...,153.213333,149.100006,327.0,308.370361,289.0,13.7,13.470967,13.3,0.529097,0.528816
347,2,2.0,93.0,0.0,45.0,10.0,0.0,45.0,10.0,31.0,...,148.841934,143.300003,317.0,301.072723,285.0,13.6,13.366667,13.1,0.532446,0.531648


In [204]:
T10_final_regression_model = finalize_model(T10_model_xgb)

In [205]:
T10_prediction = predict_model(T10_final_regression_model, data=T10_data)
#prediction.drop(['Y_Quality'], axis = 1, inplace = True)

In [206]:
T10_y_class = pd.DataFrame(T10_class, columns = ['Y_Class'])
T10_idx = T10_y_class.index
T10_y_class = T10_y_class.reset_index()
T10_prediction = pd.concat([T10_prediction, T10_y_class], axis = 1)
T10_prediction = T10_prediction.set_index('index')
T10_prediction.drop(['Y_Class'], axis = 1, inplace = True)

In [207]:
T10_prediction['Y_Class'] = 22

T10_idx = T10_prediction.index
for i in range (len(T10_prediction)) :
    if (T10_prediction.iloc[i]['prediction_label'] <= (0.525067 + 0.525213)/2) :
        T10_prediction.loc[T10_idx[i], 'Y_Class'] = 0
    elif ((0.525067 + 0.525213)/2 < T10_prediction.iloc[i]['prediction_label']) and (T10_prediction.iloc[i]['prediction_label'] < (0.534837 + 0.534951)/2) :
        T10_prediction.loc[T10_idx[i], 'Y_Class'] = 1
    elif ((0.534837 + 0.534951)/2 <= T10_prediction.iloc[i]['prediction_label']) :
        T10_prediction.loc[T10_idx[i], 'Y_Class'] = 2
T10_prediction

,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,...,X_926,X_927,X_928,X_929,X_930,X_931,X_932,X_933,prediction_label,Y_Class
index,,,,,,,,,,,,,,,,,,,,,
22,2,2.0,102.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,...,157.235489,150.899994,326.0,309.290924,290.0,13.7,13.443334,13.2,0.518345,0
23,2,2.0,102.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.518380,0
25,2,2.0,97.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,...,154.074188,150.899994,327.0,310.854553,298.0,13.7,13.454839,13.2,0.529175,1
26,2,2.0,95.0,0.0,45.0,10.0,0.0,54.0,10.0,52.0,...,151.096771,146.500000,327.0,313.618195,299.0,13.7,13.461290,13.3,0.531211,1
29,2,2.0,100.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,...,156.038712,149.100006,327.0,312.109100,298.0,13.7,13.422581,13.2,0.532009,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
591,2,1.0,87.0,0.0,45.0,10.0,0.0,50.0,10.0,52.0,...,154.777420,147.699997,326.0,313.472717,298.0,13.6,13.400000,13.2,0.530287,1
592,2,2.0,98.0,0.0,45.0,10.0,0.0,45.0,10.0,31.0,...,152.535477,147.600006,326.0,312.472717,298.0,13.6,13.361290,13.2,0.528464,1
593,2,2.0,95.0,0.0,45.0,10.0,0.0,50.0,10.0,52.0,...,155.438705,149.000000,327.0,313.363647,298.0,13.6,13.351613,13.2,0.527375,1


In [208]:
T01_prediction['Y_Class'].unique()

array([2, 0, 1], dtype=int64)

# Test

In [209]:
#44
test_data = pd.read_csv("./test.csv")
test_data

,PRODUCT_ID,TIMESTAMP,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,TEST_000,2022-09-09 2:01,T100306,T_31,2.0,94.0,0.0,45.0,10.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TEST_001,2022-09-09 2:09,T100304,T_31,2.0,93.0,0.0,45.0,11.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TEST_002,2022-09-09 8:42,T100304,T_31,2.0,95.0,0.0,45.0,11.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TEST_003,2022-09-09 10:56,T010305,A_31,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TEST_004,2022-09-09 11:04,T010306,A_31,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,TEST_305,2022-11-05 11:18,T100306,T_31,2.0,91.0,0.0,45.0,10.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
306,TEST_306,2022-11-05 16:39,T100304,T_31,2.0,96.0,0.0,45.0,11.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307,TEST_307,2022-11-05 16:47,T100306,T_31,2.0,91.0,0.0,45.0,10.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
308,TEST_308,2022-11-05 20:53,T100306,T_31,2.0,95.0,0.0,45.0,10.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [210]:
test_data['YEAR'] =  pd.to_datetime(test_data['TIMESTAMP']).dt.year
test_data['MONTH'] =  pd.to_datetime(test_data['TIMESTAMP']).dt.month
test_data['DAY'] =  pd.to_datetime(test_data['TIMESTAMP']).dt.day
test_data['HOUR'] =  pd.to_datetime(test_data['TIMESTAMP']).dt.hour
test_data['MINUTE'] =  pd.to_datetime(test_data['TIMESTAMP']).dt.minute
test_data['DAY_OF_WEEK'] =  pd.to_datetime(test_data['TIMESTAMP']).dt.day_name()

In [211]:
test_data.drop(colls, inplace = True, axis = 1)

In [212]:
test_data = test_data.drop(['MONTH', 'MINUTE', 'DAY', 'YEAR', 'DAY_OF_WEEK', 'HOUR'], axis = 1)

In [213]:
test_data.loc[test_data.PRODUCT_CODE == 'A_31','PRODUCT_CODE'] = 1
test_data.loc[test_data.PRODUCT_CODE == 'T_31','PRODUCT_CODE'] = 2
test_data.loc[test_data.PRODUCT_CODE == 'O_31','PRODUCT_CODE'] = 3

test_data.drop(['PRODUCT_ID', 'TIMESTAMP'], axis = 1, inplace = True)
test_data

,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,...,X_2862,X_2863,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871
0,T100306,2,2.0,94.0,0.0,45.0,10.0,0.0,51.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,T100304,2,2.0,93.0,0.0,45.0,11.0,0.0,45.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,T100304,2,2.0,95.0,0.0,45.0,11.0,0.0,45.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,T010305,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,183.8,467.0,444.192308,423.0,NaN,NaN,NaN,NaN,NaN,NaN
4,T010306,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,179.7,465.0,443.211539,423.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,T100306,2,2.0,91.0,0.0,45.0,10.0,0.0,51.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
306,T100304,2,2.0,96.0,0.0,45.0,11.0,0.0,45.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307,T100306,2,2.0,91.0,0.0,45.0,10.0,0.0,50.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
308,T100306,2,2.0,95.0,0.0,45.0,10.0,0.0,51.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [214]:
TT05 = test_data[(test_data['LINE'] == 'T050304')|(test_data['LINE'] == 'T050307')]
TT01 = test_data[(test_data['LINE'] == 'T010305')|(test_data['LINE'] == 'T010306')]
TT10 = test_data[(test_data['LINE'] == 'T100304')|(test_data['LINE'] == 'T100306')]

In [215]:
TT05.drop(['LINE'], axis = 1, inplace = True)
TT01.drop(['LINE'], axis = 1, inplace = True)
TT10.drop(['LINE'], axis = 1, inplace = True)

In [216]:
TT05.drop(T05_col, inplace = True, axis = 1)
TT01.drop(T01_col, inplace = True, axis = 1)
TT10.drop(T10_col, inplace = True, axis = 1)

In [68]:
TT05 = TT05.fillna(T05.mean())
TT01 = TT01.fillna(T01.mean())
TT10 = TT10.fillna(T10.mean())

In [217]:
TT05 = TT05.fillna(0)
TT01 = TT01.fillna(0)
TT10 = TT10.fillna(0)

In [218]:
TT05_prediction = predict_model(T05_final_regression_model, data=TT05)
TT01_prediction = predict_model(T01_final_regression_model, data=TT01)
TT10_prediction = predict_model(T10_final_regression_model, data=TT10)

In [219]:
TT05_y_class = pd.DataFrame(TT05, columns = ['Y_Class'])
TT05_idx = TT05_y_class.index
TT05_y_class = TT05_y_class.reset_index()
TT05_prediction = pd.concat([TT05_prediction, TT05_y_class], axis = 1)
TT05_prediction = TT05_prediction.set_index('index')
TT05_prediction.drop(['Y_Class'], axis = 1, inplace = True)

In [220]:
TT01_y_class = pd.DataFrame(TT01, columns = ['Y_Class'])
TT01_idx = TT01_y_class.index
TT01_y_class = TT01_y_class.reset_index()
TT01_prediction = pd.concat([TT01_prediction, TT01_y_class], axis = 1)
TT01_prediction = TT01_prediction.set_index('index')
TT01_prediction.drop(['Y_Class'], axis = 1, inplace = True)

In [221]:
TT10_y_class = pd.DataFrame(TT10, columns = ['Y_Class'])
TT10_idx = TT10_y_class.index
TT10_y_class = TT10_y_class.reset_index()
TT10_prediction = pd.concat([TT10_prediction, TT10_y_class], axis = 1)
TT10_prediction = TT10_prediction.set_index('index')
TT10_prediction.drop(['Y_Class'], axis = 1, inplace = True)

In [222]:
TT05_prediction['Y_Class'] = 22

TT05_idx = TT05_prediction.index
for i in range (len(TT05_prediction)) :
    if (TT05_prediction.iloc[i]['prediction_label'] <= (0.525046 + 0.525086)/2) :
        TT05_prediction.loc[TT05_idx[i], 'Y_Class'] = 0
    elif ((0.525046 + 0.525086)/2 < TT05_prediction.iloc[i]['prediction_label']) and (TT05_prediction.iloc[i]['prediction_label'] < (0.534843 + 0.535405)/2) :
        TT05_prediction.loc[TT05_idx[i], 'Y_Class'] = 1
    elif ((0.534843 + 0.535405)/2 <= TT05_prediction.iloc[i]['prediction_label']) :
        TT05_prediction.loc[TT05_idx[i], 'Y_Class'] = 2
TT05_prediction['Y_Class'].unique()

array([1, 0, 2], dtype=int64)

In [223]:
TT01_prediction['Y_Class'] = 22

TT01_idx = TT01_prediction.index
for i in range (len(TT01_prediction)) :
    if (TT01_prediction.iloc[i]['prediction_label'] <= (0.524808 + 0.525221)/2) :
        TT01_prediction.loc[TT01_idx[i], 'Y_Class'] = 0
    elif ((0.524808 + 0.525221)/2 < TT01_prediction.iloc[i]['prediction_label']) and (TT01_prediction.iloc[i]['prediction_label'] < (0.534737 + 0.535279)/2) :
        TT01_prediction.loc[TT01_idx[i], 'Y_Class'] = 1
    elif ((0.534737 + 0.535279)/2 <= TT01_prediction.iloc[i]['prediction_label']) :
        TT01_prediction.loc[TT01_idx[i], 'Y_Class'] = 2
TT01_prediction['Y_Class'].unique()

array([1, 0, 2], dtype=int64)

In [224]:
TT10_prediction['Y_Class'] = 22

TT10_idx = TT10_prediction.index
for i in range (len(TT10_prediction)) :
    if (TT10_prediction.iloc[i]['prediction_label'] <= (0.525067 + 0.525213)/2) :
        TT10_prediction.loc[TT10_idx[i], 'Y_Class'] = 0
    elif ((0.525067 + 0.525213)/2 < TT10_prediction.iloc[i]['prediction_label']) and (TT10_prediction.iloc[i]['prediction_label'] < (0.534837 + 0.534951)/2) :
        TT10_prediction.loc[TT10_idx[i], 'Y_Class'] = 1
    elif ((0.534837 + 0.534951)/2 <= TT10_prediction.iloc[i]['prediction_label']) :
        TT10_prediction.loc[TT10_idx[i], 'Y_Class'] = 2
TT10_prediction['Y_Class'].unique()

array([1, 0, 2], dtype=int64)

In [225]:
final_test = pd.concat([TT05_prediction, TT01_prediction, TT10_prediction])
final_test

,PRODUCT_CODE,X_128,X_129,X_130,X_131,X_132,X_133,X_134,X_135,X_136,...,X_924,X_925,X_926,X_927,X_928,X_929,X_930,X_931,X_932,X_933
index,,,,,,,,,,,,,,,,,,,,,
7,1,18031.0,18031.0,0.0,0.0,0.19,0.20,0.19,0.2,354.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1,18064.0,18064.0,0.0,0.0,0.19,0.20,0.19,0.2,355.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,1,0.0,0.0,3370.0,3370.0,0.21,0.22,0.21,0.2,230.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,1,295.0,295.0,0.0,0.0,0.19,0.21,0.19,0.2,60.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,1,8632.0,8632.0,0.0,0.0,0.20,0.21,0.20,0.2,249.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,169.199997,161.800003,152.251617,142.899994,326.0,313.072723,294.0,13.6,13.341935,13.2
306,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,162.300003,159.300003,154.830002,152.399994,326.0,307.796295,290.0,13.6,13.406451,13.2
307,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,169.300003,158.000000,150.909683,145.000000,323.0,304.418182,289.0,13.6,13.361290,13.2


In [226]:
final_test = final_test.sort_index(ascending=True)
final_test['Y_Class']

index
0      1
1      1
2      1
3      1
4      1
      ..
305    1
306    0
307    1
308    1
309    1
Name: Y_Class, Length: 310, dtype: int64

In [228]:
#52
sample_submission = pd.read_csv('sample_submission.csv')
sample_submission['Y_Class'] = final_test['Y_Class']
display(sample_submission.head())
sample_submission.to_csv('line3regclass0_andsep_submission.csv',index=False)

,PRODUCT_ID,Y_Class
0,TEST_000,1
1,TEST_001,1
2,TEST_002,1
3,TEST_003,1
4,TEST_004,1


In [229]:
final_test['Y_Class'].value_counts()

1    264
0     39
2      7
Name: Y_Class, dtype: int64

In [230]:
np.array(final_test['Y_Class'])

array([1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 2, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 2, 1, 2, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,